In [ ]:
#调用AutoDock Vina命令进行处理

In [ ]:
%%time
##############pdb文件前处理，准备为pdbqt文件###################

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有pdb文件

    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdb' in file:
                L.append(os.path.join(root, file))
    return L


a="prepare_receptor -r "
    

for i in file_name2('path_to_files/'):#注意会遍历文件夹下所有文件，小心存储
    name=i.split("/")[-1].split(".pdb")[0]
    
    
    print(a+i+" -o "+name+".pdbqt")
        

In [ ]:
#########################################获取受体蛋白中心坐标及各个方向长度盒子，spacing:取值统一为1.
############准备conf文件
from Bio.PDB import PDBParser
import numpy as np
import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有pdb格式文件
    
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdb' in file and "pdbqt" not in file:
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 
def calculate_receptor_box_parameters(pdb_file, box_size):
    parser = PDBParser()
    structure = parser.get_structure("receptor", pdb_file)

    # 计算受体的中心坐标
    coords = np.array([atom.get_coord() for atom in structure.get_atoms()])
    center = np.mean(coords, axis=0)

    # 计算盒子尺寸
    min_coords = np.min(coords, axis=0)
    max_coords = np.max(coords, axis=0)
    size = max_coords - min_coords

    # 计算间隔
    spacing = np.array([box_size, box_size, box_size])

    return center[0], center[1], center[2], size[0], size[1], size[2], spacing[0]


box_size = 10  # 盒子尺寸（在x、y、z方向上的相同尺寸）

b="exhaustiveness =36"#设置exhaustiveness值
c="num_modes = 9"#设置num_modes值
d="energy_range = 4"#设置energy_range值
for i in file_name2('path_to_files/'):#注意会遍历文件夹下所有文件，小心存储
    print(i)
    name=i.split("/")[-1].split(".pdb")[0]
    center_x, center_y, center_z, size_x, size_y, size_z, spacing = calculate_receptor_box_parameters(i, box_size)
    f2=open(name+".conf","w")
    receptor=i.replace(".pdb",".pdbqt")
    f2.write("receptor="+receptor+"\n")
    f2.write("center_x="+str(center_x)+"\n")
    f2.write("center_y="+str(center_y)+"\n")
    f2.write("center_z="+str(center_z)+"\n")
    f2.write("size_x="+str(size_x)+"\n")
    f2.write("size_y="+str(size_y)+"\n")
    f2.write("size_z="+str(size_z)+"\n")
    f2.write(d+"\n")
    f2.write(b+"\n")
    f2.write(c+"\n")
    f2.write("spacing = 1.0\n")
    f2.close()
    

In [ ]:
%%time
##############对接命令###################

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有pdbqt格式文件
    
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if 'pdbqt' in file:
                L.append(os.path.join(root, file))
    return L
def file_name1(file_dir): #识别路径目录下包含的所有conf格式文件
    
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if 'conf' in file:
                L.append(os.path.join(root, file))
    return L


def allfiles(conf,ligand,out,log):
    a="./vina --config "+conf+" --ligand " +ligand+" --out "+out+" >"+log
    #os.system(a)
    print(a)
f=open("paht_to_vina/v4.sh","w")#生成shell命令便于后台运行
for i in file_name1('path_to_conffiles'):#注意会遍历文件夹下所有文件，小心存储
    if "gpfconf" not in i:
        #print(i)
        for j in file_name2('path_to_ligandfiles'):
            if "PL_" not in j:
                #print(j)
                conf=i
                ligand=j
                out=conf.split("/")[-1].split(".conf")[0]+"_"+ligand.split("/")[-1].split(".")[0]+"outpt.pdbqt"
                log=conf.split("/")[-1].split(".conf")[0]+"_"+ligand.split("/")[-1].split(".")[0]+"pt.log"
                a="./vina --config "+conf+" --ligand " +ligand+" --out "+out+" >"+log
                f.write(a+"\n")
f.close()
        
    
    

    
      
    
        

In [ ]:
%%time
###################################计算两个原子之间的距离，然后推认氨基酸
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
import pandas as pd
import numpy as np
import os 
import cv2

def file_name2(file_dir): #识别路径目录下包含的所有Glycosidases.pdb文件
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if 'Glycosidases.pdb' in file:
                L.append(os.path.join(root, file))
    return L
def file_name1(file_dir): #识别路径目录下包含的所有.pdbqt文件
    
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdbqt' in file:
                L.append(os.path.join(root, file))
    return L


def calculate_distance(atom1, atom2):
    """计算两个原子之间的距离"""
    return np.linalg.norm(atom1 - atom2)

def find_residues_within_distance(pdb_file, ligand_file, distance_threshold):
    """查找在指定距离范围内的氨基酸残基"""
    parser = PDBParser(QUIET=True)
    protein_structure = parser.get_structure('protein', pdb_file)
    
    # 获取配体的原子坐标
    ligand_parser = PDBParser(QUIET=True)
    ligand_structure = ligand_parser.get_structure('ligand', ligand_file)
    #ligand_atoms = list(ligand_structure.get_atoms())
    
    residues_within_distance ={}
    ##########################################注意，此处均为单链蛋白##############
    
    for modelligand in ligand_structure:
        residues_within_distance[str(modelligand.id+1)]=[]
        ligand_atoms = list(modelligand.get_atoms())
        
        for model in protein_structure:
            for chain in model:
                for residue in chain:
                    flag=0
                    for atom in residue:
                        if flag==1:break
                        for ligand_atom in ligand_atoms:
                            distance = calculate_distance(atom.get_coord(), ligand_atom.get_coord())
                            
                            if distance <= distance_threshold:
                                #print(ligand_atom.get_coord())
                                residues_within_distance[str(modelligand.id+1)].append(residue)
                                flag=1
                                break
                                
                                
    
    return residues_within_distance

# 设置参数
pdb_files = 'path_to_pdb_files/'  # 蛋白结构的PDB文件
ligand_files = 'path_to_outligandfiles/'  # 配体构型的PDBQT文件
distance_threshold = 5.0  # 距离阈值
f=open("anjisuan33.csv","w")
for pdb_file in file_name2(pdb_files):
    print(pdb_file)
    for ligand_file in file_name1(ligand_files):
        if "3"==ligand_file.split("/")[-1][0]:
            key=ligand_file.split("/")[-1].split("Glycosidases")[0]
        else: key=ligand_file.split("/")[-1].split("Glycosidases")[0]
        #key=ligand_file.split("/")[-1].split("_")[1]
        if key in pdb_file:
            #print(ligand_file.split("/")[-1])
            anjisuan=find_residues_within_distance(pdb_file, ligand_file, distance_threshold)
            
            for key in anjisuan:
                
                f.write(pdb_file.split("/")[-1]+","+ligand_file.split("/")[-1]+","+str(key))
                if len(anjisuan[key])!=0:
                    for kk in anjisuan[key]:
                        f.write(","+kk.get_resname()+"_"+str(kk.get_id()[1]))
                else:f.write(",NA")
                f.write("\n")

    
f.close()            
    
    



In [ ]:
%%time
####################统计各位点出现次数
f=open("anjisuan33.csv","r")
dictseq={}
while 1:
    a=f.readline().strip()
    if a=="":break
    b=a.split(",")
    name=b[0]
    if name not in dictseq:
        dictseq[name]={}
        i=3
        while i<len(b):
            if b[i] not in dictseq[name]:
                dictseq[name][b[i]]=1
            else:dictseq[name][b[i]]+=1
            i+=1
    else:
        i=3
        while i<len(b):
            if b[i] not in dictseq[name]:
                dictseq[name][b[i]]=1
            else:dictseq[name][b[i]]+=1
            i+=1

f.close()

f=open("anjisuanxulieaa.csv","w")
for key1 in dictseq:
    f.write(key1)
    for key0 in sorted(dictseq[key1].items(), key=lambda item:item[1], reverse=True):
        f.write(","+key0[0]+","+str(key0[1]))
    f.write("\n")
f.close()

In [ ]:
%%time
###################################提取出得到的活性中心结构
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
import pandas as pd
import numpy as np
import os 
import cv2
import re

def file_name2(file_dir): #识别路径目录下包含的所有png格式照片
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdb' in file and '.pdbqt' not in file:
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 


pdb_files = 'path_to_pdb_files/'  # 蛋白结构的PDB文件

f=open("anjisuanxulieaa.csv","r")
dictseq={}
while 1:
    a=f.readline().strip()
    if a=="":break
    b=a.split(",")
    dictseq[b[0]]={}
    #print(b[0])
    i=2
    #print(len(b))
    while i<len(b):
        #print(i)
        if int(b[i])>9 and "NA" not in b[i-1]:############################筛选条件，此次计算中大于9次以上的位点
            dictseq[b[0]][b[i-1].split("_")[1]]=b[i-1].split("_")[0]
        i+=2

f.close()
print(len(dictseq))
for pdb_file in file_name2(pdb_files):
    pdb=pdb_file.split("/")[-1]
    if pdb in dictseq:
        print(pdb)
        fo=open(pdb_file,"r")
        f=open("path_to_pdb_files/canji/"+pdb.replace(".pdb","canji.pdbqt"),"w")
        while 1:
            a=fo.readline().strip()
            if a=="":break
            b = re.split(r"\s+", a)
            if len(b)>5 and b[5] in dictseq[pdb]:
                if b[3] !=dictseq[pdb][b[5]]:print(a)
                f.write(a+"\n")
            if "TER" in a[:5]:
                f.write(a+"\n")
        fo.close()
        f.close()
    
            
